In [1]:
import os
import clr
import sys
from imp import reload
from System import Array
from time import sleep, time
import numpy as np
from matplotlib import pyplot as plt
from scipy.misc import factorial
import peakutils
# sys.path.append(os.getcwd())
%matplotlib inline

import qcodes as qc
qc.show_subprocess_widget()

import qcodes.instrument.parameter as parameter

loc_provider = qc.data.location.FormatLocation(fmt='data/{date}/#{counter}_{name}_{time}')
qc.data.data_set.DataSet.location_provider=loc_provider

<IPython.core.display.Javascript object>

In [2]:
dummy = parameter.ManualParameter(name="dummy")

def try_close_instruments(instruments=['PulseBlaster', 'ArbStudio', 'SIM900', 'ATS', 'ATS_controller']):
    if isinstance(instruments, str):
        instruments = [instruments]
    for instrument in instruments:
        try:
            eval('{}.close()'.format(instrument_name))
        except:
            pass
        
def configure_PulsBlaster(PulseBlaster, repetition_rate, marker_cycles=100, sampling_rate=500):
    cycles = sampling_rate*1e6 * repetition_rate

    PulseBlaster.detect_boards()
    PulseBlaster.select_board(0)
    PulseBlaster.core_clock(sampling_rate)

    PulseBlaster.start_programming()
    start = PulseBlaster.send_instruction(1,'continue',0,marker_cycles)
    PulseBlaster.send_instruction(0, 'branch', start, cycles - marker_cycles)
    PulseBlaster.stop_programming()
    
def configure_ArbStudio(ArbStudio, repetition_rate, marker_cycles=100):
    ArbStudio.ch3_trigger_mode('stepped')
    ArbStudio.ch3_trigger_source('fp_trigger_in')

    # Add waveforms
    ArbStudio.ch3_clear_waveforms()
    ArbStudio.ch3_add_waveform([1]*marker_cycles + [0,0])

    #Add sequence
    ArbStudio.ch3_sequence([0])

    #Load waveforms and sequence
    waveforms = ArbStudio.load_waveforms()
    sequences = ArbStudio.load_sequence()

In [3]:
import qcodes.instrument_drivers.spincore.PulseBlasterESRPRO as PulseBlaster_driver
import qcodes.instrument_drivers.lecroy.ArbStudio1104 as ArbStudio_driver
import qcodes.instrument_drivers.stanford_research.SIM900 as SIM900_driver
import qcodes.instrument_drivers.AlazarTech.ATS9440 as ATS_driver
import qcodes.instrument_drivers.AlazarTech.ATS_acquisition_controllers as ATS_controller_driver

for instrument_name in ['PulseBlaster', 'ArbStudio', 'SIM900', 'ATS', 'ATS_controller']:
    try_close_instruments(instrument_name)
    eval('reload({}_driver)'.format(instrument_name))

PulseBlaster = PulseBlaster_driver.PulseBlaster('PulseBlaster')

dll_path = os.path.join(os.getcwd(),'lecroy_driver\\Library\\ArbStudioSDK.dll')
ArbStudio = ArbStudio_driver.ArbStudio1104('ArbStudio', dll_path, server_name=None)

SIM900 = SIM900_driver.SIM900('SIM900', 'GPIB0::4::INSTR')
TG, LB, RB, TGAC, SRC, _, DS, DF = [eval('SIM900.chan{}'.format(i)) for i in range(1,9)]

ATS = ATS_driver.ATS9440('ATS', server_name='Alazar_server')
ATS_controller = ATS_controller_driver.Average_AcquisitionController(name='ATS_control', 
                                                           alazar_name='ATS',
                                                           server_name='Alazar_server')
# Configure ATS and ATS_controller
ATS.config(trigger_source1='CHANNEL_C',
           trigger_level1=135,
           channel_range=2,
           sample_rate=1e6,
           coupling='DC')

In [4]:
repetition_rate = 0.01

configure_PulsBlaster(PulseBlaster, repetition_rate)
configure_ArbStudio(ArbStudio, repetition_rate)
ATS_controller.average_mode('point')
samples_per_record = int(16 * round(float(ATS.sample_rate()*repetition_rate*0.7)/16))
ATS_controller.update_acquisition_kwargs(buffer_timeout=5000,
                                         samples_per_record=samples_per_record,
                                         records_per_buffer=1,
                                         buffers_per_acquisition=1,
                                         channel_selection='A')

In [8]:
ArbStudio.stop()
PulseBlaster.stop()

qc.halt_bg()

No loop running


In [9]:
ArbStudio.run([3])
PulseBlaster.start()
data = qc.Loop(TGAC[1:2:0.01], delay=0.001).loop(DS[2:3:0.01]).each(qc.Task(DF,DS),
    ATS_controller.acquisition).run(name='testsweep', progress_interval=True)

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = 'data/2016-08-20/#070_testsweep_17-04-14'
   <Type>   | <array_id>                   | <array.name>     | <array.shape>
   Setpoint | SIM900_chan4_set             | chan4            | (100,)
   Setpoint | SIM900_chan7_set             | chan7            | (100, 100)
   Measured | ATS_control_Channel_A_signal | Channel_A_signal | (100, 100)
started at 2016-08-20 17:04:14


In [ ]:
data = qc.Loop(dummy(1:, delay=0.01).each(
    ATS_controller.acquisition).run(name='testsweep', progress_interval=True)